![Banner](../../assets/banner-sprint7-desafio.png)



## IMPORTAÇÕES E LEITURA DO DATASET

Importação de bibliotecas para análise e tratamento dos dados.

In [5]:
import pandas as pd
import re
#from transformers import pipeline

Leitura do dataset para um DataFrame, com configurações específicas para o formato.



In [40]:
df = pd.read_csv('../../sprint6/desafio/dados_raw/movies.csv',
    delimiter='|',
    quotechar='"',
    escapechar='\\',
    engine='python'
)

Visualização de amostra.

In [ ]:
df.head()

### Filtro 1: Gênero

Dentre os gêneros designados, foi decidido manter somente `Romance` para um recorte mais específico.

In [41]:
df1 = df[df['genero'].str.contains(r'^Romance$', regex=True)]
df1.shape

(11919, 15)

### Filtro 2: Ano 2013 ~ Atual

Recorte referente ao período de popularização da Netflix, abarcando o início de suas atividades fora dos EUA, a expansão e presença global, até os dias atuais.

Primeiramente, foi realizada a tratativa de possíveis valores nulos na coluna `anoLancamento` e, depois, conversão da coluna para `int` .

In [42]:
df1.loc[df1['anoLancamento'].str.contains(r'^[a-zA-Z]+$', regex=True), 'anoLancamento'] = 0
df1['anoLancamento'] = df1['anoLancamento'].astype(int)

C:\Users\Intel\AppData\Local\Temp\ipykernel_13664\1949478750.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['anoLancamento'] = df1['anoLancamento'].astype(int)


Confirmação da conversão.

In [ ]:
df1.info()

Recorte por período, este DataFrame será a base para inserção dos dados obtidos via API, visto que serão utilizadas as linhas com atrizes/atores distintos para a análise.

In [43]:
df2 = df1[df1['anoLancamento'] >= 2013]
df2.shape

(3975, 15)

Download deste DataFrame para utilização futura.

In [21]:
df2.to_csv(
    './filmes_filtrado_com_elenco.csv',
    sep=',',
    header=True,
    index=False
)

### Filtro 3: Filtrar Filmes Distintos

Este é o filtro utilizado para obter somente os IDs distintos dos filmes interesse para as requisições via API.

In [22]:
df3 = df2.drop_duplicates(subset=['id'], keep='last')
df3

,id,tituloPincipal,tituloOriginal,anoLancamento,tempoMinutos,genero,notaMedia,numeroVotos,generoArtista,personagem,nomeArtista,anoNascimento,anoFalecimento,profissao,titulosMaisConhecidos
563534,tt0850678,Jackson Arms,Jackson Arms,2015,90,Romance,6.7,20,actress,Grace Jiang,Kim Jiang Dubaniewicz,N,N,"actress,director,producer","tt9554862,tt2384999,tt0472196,tt7820648"
576817,tt10006510,Life Mein Twist Hai,Life Mein Twist Hai,2014,N,Romance,9.6,8,actress,N,Arshi Suryavanshi,N,N,NaN,N
577446,tt10033852,Rangeela Raybaa,Rangeela Raybaa,2017,143,Romance,8.3,18,actor,N,Alhad Arvind Andore,N,N,actor,N
577449,tt10033904,Spandan What is Relationship,Spandan What is Relationship,2017,142,Romance,7.4,5,actor,N,Shailesh Narayan Kulkarni,N,N,"director,actor,producer",N
577569,tt10039472,Premak Basaat,Premak Basaat,2018,N,Romance,8.9,10,actor,Birju Kaka,Rajeev Jha,N,N,actor,"tt19868542,tt1886663,tt10039472,tt14091818"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1042395,tt9743420,First Love,Tinh dau tho ngay,2019,85,Romance,5.1,18,actress,My An,Lam Thanh My,N,N,actress,"tt4777250,tt3595748,tt2724790,tt4911780"
1042691,tt9765480,Elise,Elise,2019,105,Romance,7.4,87,actress,Elise,Janine Gutierrez,1989,N,actress,"tt11156674,tt2222500,tt5890698,tt14766058"
1042761,tt9766610,Cho Em Gan Anh Them Chut Nua,Cho Em Gan Anh Them Chut Nua,2016,96,Romance,5.6,17,actor,N,Vo Dinh Hieu,N,N,actor,"tt15173988,tt9766610,tt5994790,tt11242932"
1043920,tt9840958,Love Struck Sick,Love Struck Sick,2019,92,Romance,5.6,3183,actor,Tim Gayland Fan,Jim Averill,N,N,"cinematographer,actor,producer","tt13284126,tt9840958,tt8018008,tt21213644"


#### Inserção de Coluna de Título Original em Inglês (Boolean)

Tentativa de filtrar excluindo as linhas com títulos originais em inglês, a partir de utilização de modelo de linguagem para reconhecimento de língua.

Apesar da classificação correta, alguns títulos da coluna `tituloOriginal` não estão na língua correta, e sim em inglês.

Portanto, este filtro foi inutilizado.

In [9]:
detector_lingua = pipeline("text-classification",
                           model="papluca/xlm-roberta-base-language-detection")

def ingles(texto):
    classificacao = detector_lingua(texto)
    return classificacao[0]['label'] == 'en'

df3['emIngles'] = df3['tituloOriginal'].apply(ingles)
df3

Device set to use cpu
/tmp/ipykernel_2497/3497210258.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3['Is_English'] = df3['tituloOriginal'].apply(is_english)


,id,tituloPincipal,tituloOriginal,anoLancamento,tempoMinutos,genero,notaMedia,numeroVotos,generoArtista,personagem,nomeArtista,anoNascimento,anoFalecimento,profissao,titulosMaisConhecidos,Is_English
134122,tt0062336,The Tango of the Widower and Its Distorting Mi...,El Tango del Viudo y Su Espejo Deformante,2020,70,Drama,6.4,163,actor,Silva,Luis Alarcón,1929,N,"actor,special_effects","tt3234226,tt0379108,tt0242198,tt1947646",False
152297,tt0069049,The Other Side of the Wind,The Other Side of the Wind,2018,122,Drama,6.7,7455,actress,The Actress,Oja Kodar,1941,N,"actress,writer,director","tt0108506,tt0239070,tt0209994,tt0069049",True
266695,tt0120589,A Dangerous Practice,A Dangerous Practice,2022,108,Drama,7.0,135,actor,Cícero,Cássio Gabus Mendes,1961,N,"actor,producer","tt0149517,tt0478978,tt1587828,tt0840797",True
317301,tt0173042,Premaloka,Premaloka,2021,154,"Drama,Romance",7.9,518,actor,Ravi,V. Ravichandran,1961,N,"actor,director,writer","tt0173189,tt0173042,tt0308793,tt0470192",False
339910,tt0192528,Heaven & Hell,Reverse Heaven,2018,104,Drama,3.9,118,actor,Demon in Heaven,Tim Dax,N,N,"actor,producer,director","tt15203302,tt3796746,tt4997978,tt5225574",True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1045093,tt9911774,Padmavyuhathile Abhimanyu,Padmavyuhathile Abhimanyu,2019,130,Drama,8.0,378,actor,N,Anoop Chandran,N,N,"actor,producer","tt1028558,tt0867145,tt2429086,tt9244322",False
1045101,tt9913660,No Apology,No Apology,2019,102,Drama,5.7,66,actress,Young Robin,Kiara Corkett,N,N,actress,"tt6588762,tt5950436,tt9913660,tt6056808",True
1045121,tt9914942,Life Without Sara Amat,La vida sense la Sara Amat,2019,74,Drama,6.7,164,actress,Sara,Maria Morera,N,N,actress,"tt10589738,tt9914942,tt14604096,tt14596388",False
1045134,tt9916170,The Rehearsal,O Ensaio,2019,51,Drama,7.0,7,actress,N,Julia Ianina,1983,N,"actress,writer","tt5822138,tt12329148,tt18309812,tt0293007",False


In [11]:
df3.to_csv(
    './filmes_distintos_attr_em_ingles.csv',
    sep=',',
    header=True,
    index=False
)